In [1]:
import pandas as pd
import re

In [2]:
mtdata = pd.read_csv('mousetrackercorrected.csv')

In [3]:
includelist = pd.read_csv('n=452 subjectID.csv', header = None)
includelist = includelist[0].values

In [4]:
mtdata.loc[mtdata['subject'].isin(includelist)]

,subject,trial,stimfile,condition,code,resp_1,resp_2,response,distractor,error,...,Y_92,Y_93,Y_94,Y_95,Y_96,Y_97,Y_98,Y_99,Y_100,Y_101


In [5]:
mtdata['RESPONSE1'] = [x for x in mtdata['resp_1'].str.extract(r'..(\d*)_.(\d*)', expand = True).values]
mtdata['RESPONSE2'] = [x for x in mtdata['resp_2'].str.extract(r'..(\d*)_.(\d*)', expand = True).values]

In [6]:
mtdata.head()

,subject,trial,stimfile,condition,code,resp_1,resp_2,response,distractor,error,...,Y_94,Y_95,Y_96,Y_97,Y_98,Y_99,Y_100,Y_101,RESPONSE1,RESPONSE2
0,455806,1,NaN,777,prac2,~S3_O6.jpg,~S7_O8.jpg,1,1,1,...,1.1400,1.1400,1.1400,1.1400,1.1400,1.1400,1.1400,1.1400,"[3, 6]","[7, 8]"
1,455806,3,NaN,777,prac8,~S5_O4.jpg,~S10_O11.jpg,1,1,1,...,1.1500,1.1500,1.1500,1.1500,1.1500,1.1500,1.1500,1.1500,"[5, 4]","[10, 11]"
2,455806,5,NaN,777,prac4,~S5_O4.jpg,~S8_O9.jpg,1,2,0,...,1.1250,1.1250,1.1250,1.1250,1.1250,1.1250,1.1250,1.1250,"[5, 4]","[8, 9]"
3,455806,7,NaN,777,prac5,~S10_O9.jpg,~S7_O8.jpg,1,1,1,...,1.2125,1.2125,1.2125,1.2125,1.2125,1.2125,1.2125,1.2125,"[10, 9]","[7, 8]"
4,455806,9,NaN,777,smgmvsslgl12,~S14_O10.jpg,~S9_O12.jpg,1,2,0,...,1.1900,1.1900,1.1900,1.1900,1.1900,1.1900,1.1900,1.1900,"[14, 10]","[9, 12]"


if error == 0, then selfish, and that's response 1?
if 1 then altruistic

ERROR 0 REPSPONSE 0
ERROR 1 RESPONSE 0
ERROR 0 RESPONSE 1
ERROR 1 RESPONSE 1

In [7]:
tempdata = pd.DataFrame(columns = ('RESPONSE','ERROR','RESPONSE1','RESPONSE2'))
tempdata['RESPONSE'] = mtdata['response']
tempdata['ERROR'] = mtdata['error']
tempdata['RESPONSE1'] = mtdata['RESPONSE1']
tempdata['RESPONSE2'] = mtdata['RESPONSE2']
tempdata['SELFISHCHOICESELFMORE'] = 0
tempdata['SELFISHCHOICEGROUPMORE'] = 0

In [8]:
tempdata.head()

,RESPONSE,ERROR,RESPONSE1,RESPONSE2,SELFISHCHOICESELFMORE,SELFISHCHOICEGROUPMORE
0,1,1,"[3, 6]","[7, 8]",0,0
1,1,1,"[5, 4]","[10, 11]",0,0
2,1,0,"[5, 4]","[8, 9]",0,0
3,1,1,"[10, 9]","[7, 8]",0,0
4,1,0,"[14, 10]","[9, 12]",0,0


In [9]:
SELFISHCHOICESELFMORE = []
SELFISHCHOICEGROUPMORE = []

for row in tempdata.iterrows():
    if (row[1][0] == 1) & (row[1][1] == 0) | ((row[1][0] == 2) & (row[1][1] == 1)):
        try:
            SELFISHCHOICESELFMORE.append(int(row[1][2][0]) - int(row[1][3][0]))
            SELFISHCHOICEGROUPMORE.append((int(row[1][2][0]) + int(row[1][2][1])) - (int(row[1][3][0]) + int(row[1][3][1])))
        except:
            SELFISHCHOICESELFMORE.append(None)
            SELFISHCHOICEGROUPMORE.append(None)
    elif ((row[1][0] == 2) & (row[1][1] == 0)) | ((row[1][0] == 1) & (row[1][1] == 1)):
        try:
            SELFISHCHOICESELFMORE.append(int(row[1][3][0]) - int(row[1][2][0]))
            SELFISHCHOICEGROUPMORE.append((int(row[1][3][0]) + int(row[1][3][1])) - (int(row[1][2][0]) + int(row[1][2][1])))
        except:
            SELFISHCHOICESELFMORE.append(None)
            SELFISHCHOICEGROUPMORE.append(None)

In [10]:
tempdata = tempdata.drop(['RESPONSE1','RESPONSE2', 'RESPONSE', 'ERROR'], axis = 1)
tempdata['SELFISHCHOICESELFMORE'] = SELFISHCHOICESELFMORE
tempdata['SELFISHCHOICEGROUPMORE'] = SELFISHCHOICEGROUPMORE

In [11]:
outdata = pd.concat([mtdata,tempdata], axis = 1)

In [13]:
outdata.to_csv('mousetrackercorrected9.16.2016.csv')